## Problemstellung

Eine Bank möchte die **Kreditwürdigkeit** ihrer Kunden vorhersagen.  
Auf Basis von Kundendaten (z. B. Kontostatus, Kredithistorie, Einkommen, Beschäftigung, Alter, usw.) soll ein Vorhersagemodell entwickelt werden, das jeden Kunden in eine der beiden Klassen einordnet:

- **kreditwürdig (1)**  
- **nicht kreditwürdig (2)**  

Besonderheit ist das von der Bank vorgegebene **Kostenmodell**:  

- Einen *nicht kreditwürdigen Kunden fälschlich als kreditwürdig* einzustufen, verursacht **fünfmal höhere Kosten** als den umgekehrten Fehler (*kreditwürdigen Kunden fälschlich als nicht kreditwürdig*).  

Daraus ergibt sich ein **überwachtes Klassifikationsproblem mit asymmetrischen Fehlkosten**.  

- **Zielvariable (Label):** `Creditworthy`  
- **Eingabeattribute:** Kundendaten wie  
  - Status of existing checking account  
  - Duration  
  - Credit history  
  - Purpose  
  - Credit amount  
  - Employment  
  - Job  
  - Foreign worker  
  - usw.


In [2]:
# Cell 1: Load data, rename columns, overview
import pandas as pd
import numpy as np

# Datei einlesen
file_path = "kredit.dat"
# Deine Originalzeile (zeigt eine FutureWarning, funktioniert aber):
df = pd.read_csv(file_path, delim_whitespace=True, header=None)
# Alternative ohne FutureWarning:
# df = pd.read_csv(file_path, sep=r"\s+", header=None, engine="python")

# Spaltennamen laut Projektbeschreibung
df.columns = [
    "Status_checking_account",   # 0
    "Duration_months",           # 1
    "Credit_history",            # 2
    "Purpose",                   # 3 (incomplete)
    "Credit_amount",             # 4
    "Savings_account_bonds",     # 5
    "Employment",                # 6 (incomplete)
    "Installment_rate",          # 7
    "Personal_status_sex",       # 8
    "Other_debtors_guarantors",  # 9
    "Present_residence_since",   # 10
    "Property",                  # 11
    "Age_years",                 # 12
    "Other_installment_plans",   # 13
    "Housing",                   # 14
    "Number_credits_bank",       # 15
    "Job",                       # 16 (incomplete)
    "People_liable",             # 17
    "Telephone",                 # 18
    "Foreign_worker",            # 19 (incomplete)
    "Creditworthy"               # 20 (target)
]

# "?" als fehlende Werte behandeln
df.replace("?", np.nan, inplace=True)

# Übersicht
print("Shape:", df.shape)
print("\nFehlende Werte je Spalte:\n", df.isnull().sum())

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
display(df.head())


Shape: (1000, 21)

Fehlende Werte je Spalte:
 Status_checking_account       0
Duration_months               0
Credit_history                0
Purpose                     173
Credit_amount                 0
Savings_account_bonds         0
Employment                  496
Installment_rate              0
Personal_status_sex           0
Other_debtors_guarantors      0
Present_residence_since       0
Property                      0
Age_years                     0
Other_installment_plans       0
Housing                       0
Number_credits_bank           0
Job                         232
People_liable                 0
Telephone                     0
Foreign_worker              360
Creditworthy                  0
dtype: int64


C:\Users\moham\AppData\Local\Temp\ipykernel_16152\995430969.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, header=None)


,Status_checking_account,Duration_months,Credit_history,Purpose,Credit_amount,Savings_account_bonds,Employment,Installment_rate,Personal_status_sex,Other_debtors_guarantors,Present_residence_since,Property,Age_years,Other_installment_plans,Housing,Number_credits_bank,Job,People_liable,Telephone,Foreign_worker,Creditworthy
0,A14,36,A32,NaN,2299,A63,NaN,4,A93,A101,4,A123,39,A143,A152,1,A173,1,A191,NaN,1
1,A12,18,A32,A46,1239,A65,A73,4,A93,A101,4,A124,61,A143,A153,1,NaN,1,A191,A201,1
2,A13,24,A32,A40,947,A61,A74,4,A93,A101,3,A124,38,A141,A153,1,NaN,2,A191,NaN,2
3,A14,15,A33,A43,1478,A61,A73,4,A94,A101,3,A121,33,A141,A152,2,A173,1,A191,A201,1
4,A14,24,A32,A40,1525,A64,A74,4,A92,A101,3,A123,34,A143,A152,1,A173,2,A192,A201,1


In [ ]:
# Cell 2: Numeric EDA (stats + outliers) and plots
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np

# Numerische Spalten
numeric_cols = [
    "Duration_months",
    "Credit_amount",
    "Installment_rate",
    "Present_residence_since",
    "Age_years",
    "Number_credits_bank",
    "People_liable"
]

# Numerisch casten
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Basisstatistiken
print("\n=== Basis-Statistiken numerischer Variablen ===")
print(df[numeric_cols].describe().T)

# IQR-Ausreißergrenzen
def iqr_outlier_bounds(s: pd.Series):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return lower, upper

print("\n=== Grobe Ausreißerzählung (IQR-Regel) ===")
for col in numeric_cols:
    s = df[col].dropna()
    if s.empty:
        print(f"{col}: keine Werte")
        continue
    lo, up = iqr_outlier_bounds(s)
    n_low = (s < lo).sum()
    n_high = (s > up).sum()
    print(f"{col}: unten={n_low}, oben={n_high}, (Schranken: {lo:.2f}, {up:.2f})")

# Verzeichnis für Plots
out_dir = Path("figures of analysis")
out_dir.mkdir(exist_ok=True)

# Histogramme & Boxplots pro numerischer Spalte
for col in numeric_cols:
    s = df[col].dropna()

    # Histogramm
    plt.figure()
    s.hist(bins=30)
    plt.title(f"Histogramm: {col}")
    plt.xlabel(col)
    plt.ylabel("Häufigkeit")
    plt.tight_layout()
    plt.savefig(out_dir / f"hist_{col}.png", dpi=150)
    plt.close()

    # Boxplot
    plt.figure()
    plt.boxplot(s, vert=True, whis=1.5)
    plt.title(f"Boxplot: {col}")
    plt.ylabel(col)
    plt.tight_layout()
    plt.savefig(out_dir / f"box_{col}.png", dpi=150)
    plt.close()

print(f"\nPlots gespeichert unter: {out_dir.resolve()}")



=== Basis-Statistiken numerischer Variablen ===
                          count      mean          std    min     25%     50%  \
Duration_months          1000.0    20.903    12.058814    4.0    12.0    18.0   
Credit_amount            1000.0  3271.258  2822.736876  250.0  1365.5  2319.5   
Installment_rate         1000.0     2.973     1.118715    1.0     2.0     3.0   
Present_residence_since  1000.0     2.845     1.103718    1.0     2.0     3.0   
Age_years                1000.0    35.546    11.375469   19.0    27.0    33.0   
Number_credits_bank      1000.0     1.407     0.577654    1.0     1.0     1.0   
People_liable            1000.0     1.155     0.362086    1.0     1.0     1.0   

                             75%      max  
Duration_months            24.00     72.0  
Credit_amount            3972.25  18424.0  
Installment_rate            4.00      4.0  
Present_residence_since     4.00      4.0  
Age_years                  42.00     75.0  
Number_credits_bank         2.00      

In [ ]:
# Cell 3: Missing values bar plot + percentages
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

out_dir = Path("figures")
out_dir.mkdir(exist_ok=True)

# Fehlende Werte – absolute Zahlen
missing_counts = df.isna().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)
print("\n=== Fehlende Werte je Spalte (>0) ===")
print(missing_counts)

# Balkendiagramm speichern (falls vorhanden)
if not missing_counts.empty:
    plt.figure()
    missing_counts.plot(kind="bar")
    plt.title("Fehlende Werte je Spalte")
    plt.xlabel("Spalte")
    plt.ylabel("Anzahl fehlender Werte")
    plt.tight_layout()
    plt.savefig(out_dir / "missing_values_bar.png", dpi=150)
    plt.close()
    print(f"Balkendiagramm gespeichert: {out_dir / 'missing_values_bar.png'}")
else:
    print("Keine fehlenden Werte vorhanden.")

# Fehlende Werte – Prozent
missing_pct = (df.isna().mean() * 100).round(2)



=== Fehlende Werte je Spalte (>0) ===
Employment        496
Foreign_worker    360
Job               232
Purpose           173
dtype: int64
Balkendiagramm gespeichert: figures\missing_values_bar.png

=== Anteil fehlender Werte in % ===
Employment        49.6
Foreign_worker    36.0
Job               23.2
Purpose           17.3
dtype: float64


In [8]:
# Numeric cast
numeric_cols = [
    "Duration_months","Credit_amount","Installment_rate",
    "Present_residence_since","Age_years","Number_credits_bank","People_liable"
]
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Target as int (not used for imputation eval, but needed later for modeling)
df["Creditworthy"] = pd.to_numeric(df["Creditworthy"], errors="coerce").astype("Int64")

# Categorical cols (everything else except numeric + target)
categorical_cols = [c for c in df.columns if c not in numeric_cols + ["Creditworthy"]]

# Columns we must impute (categorical classification)
to_impute_cat = ["Purpose", "Employment", "Job", "Foreign_worker"]

# Cell 2: Classification imputation with evaluation (precision/recall/F1/accuracy)
from typing import List, Tuple, Dict
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import numpy as np

def build_clf_preprocessor(feat_cats: List[str], feat_nums: List[str]) -> ColumnTransformer:
    """
    Preprocess features for a linear classifier that imputes a categorical column:
    - Categorical features: simple-impute most_frequent (only as helper for features), then OneHot
    - Numeric features: simple-impute median (only as helper for features), then StandardScaler
    NOTE: This SimpleImputer is only to make the *features* trainable while other targets still have NaNs.
    The imputed column itself is predicted by the classifier, not by SimpleImputer.
    """
    return ColumnTransformer(
        transformers=[
            ("cat", Pipeline(steps=[
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
            ]), feat_cats),
            ("num", Pipeline(steps=[
                ("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler())
            ]), feat_nums),
        ],
        remainder="drop"
    )

def evaluate_classification_imputation(
    df: pd.DataFrame,
    target_cat: str,
    categorical_features: List[str],
    numeric_features: List[str],
    mask_frac: float = 0.2,
    n_splits: int = 5,
    random_state: int = 42
) -> Dict[str, float]:
    """
    Evaluate supervised imputation quality for one categorical column using masking + CV.
    Returns dict with mean metrics across folds: accuracy, precision_macro, recall_macro, f1_macro.
    """
    rng = np.random.default_rng(random_state)

    # rows where target_cat is known (we can mask them)
    known_mask = df[target_cat].notna()
    idx_known = df.index[known_mask]

    # prepare arrays to collect metrics
    accs, precs, recs, f1s = [], [], [], []

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    y_known = df.loc[idx_known, target_cat].astype(str).values

    for train_idx, test_idx in skf.split(idx_known, y_known):
        # Within each fold, further mask a fraction of the TRAIN knowns and predict them
        idx_train_known = idx_known[train_idx]
        y_train_known = df.loc[idx_train_known, target_cat].astype(str).values

        # choose which portion to mask in train-known
        m = max(1, int(mask_frac * len(idx_train_known)))
        masked_indices = rng.choice(idx_train_known, size=m, replace=False)

        # copy df for this fold
        df_fold = df.copy()
        # mask target in masked_indices
        df_fold.loc[masked_indices, target_cat] = np.nan

        # training data = rows where target not missing (excluding masked part)
        mask_train = df_fold[target_cat].notna()
        # prediction set = masked_indices
        mask_pred = df_fold[target_cat].isna()

        feat_cats = [c for c in categorical_features if c != target_cat]
        feat_nums = numeric_features

        X_train = df_fold.loc[mask_train, feat_cats + feat_nums]
        y_train = df_fold.loc[mask_train, target_cat].astype(str)
        X_pred  = df_fold.loc[mask_pred, feat_cats + feat_nums]
        y_true  = df.loc[mask_pred, target_cat].astype(str)  # original truth

        pre = build_clf_preprocessor(feat_cats, feat_nums)
        clf = LogisticRegression(max_iter=1000, class_weight="balanced")

        pipe = Pipeline(steps=[("pre", pre), ("clf", clf)])
        pipe.fit(X_train, y_train)
        y_hat = pipe.predict(X_pred)

        accs.append(accuracy_score(y_true, y_hat))
        precs.append(precision_score(y_true, y_hat, average="macro", zero_division=0))
        recs.append(recall_score(y_true, y_hat, average="macro", zero_division=0))
        f1s.append(f1_score(y_true, y_hat, average="macro", zero_division=0))

    return {
        "accuracy": float(np.mean(accs)),
        "precision_macro": float(np.mean(precs)),
        "recall_macro": float(np.mean(recs)),
        "f1_macro": float(np.mean(f1s)),
        "n_splits": n_splits,
        "mask_frac": mask_frac
    }

# Run evaluation for each categorical column with missing values
results = {}
for col in to_impute_cat:
    print(f"\nEvaluating imputation for: {col}")
    res = evaluate_classification_imputation(
        df=df,
        target_cat=col,
        categorical_features=categorical_cols,
        numeric_features=numeric_cols,
        mask_frac=0.2,   # 20% artificial masking
        n_splits=5,
        random_state=42
    )
    results[col] = res
    print(res)

pd.DataFrame(results).T




Evaluating imputation for: Purpose
{'accuracy': 0.09704918032786884, 'precision_macro': 0.08237070221960327, 'recall_macro': 0.17099983598552168, 'f1_macro': 0.10279254461862719, 'n_splits': 5, 'mask_frac': 0.2}

Evaluating imputation for: Employment
{'accuracy': 0.048263888888888884, 'precision_macro': 0.03769015495141944, 'recall_macro': 0.31956206059115877, 'f1_macro': 0.06548412693052383, 'n_splits': 5, 'mask_frac': 0.2}

Evaluating imputation for: Job
{'accuracy': 0.15520331025702236, 'precision_macro': 0.0974009225368424, 'recall_macro': 0.3204684299723202, 'f1_macro': 0.1436376575564343, 'n_splits': 5, 'mask_frac': 0.2}

Evaluating imputation for: Foreign_worker
{'accuracy': 0.18051948051948052, 'precision_macro': 0.0789625388147481, 'recall_macro': 0.467868843069874, 'f1_macro': 0.1284458171360105, 'n_splits': 5, 'mask_frac': 0.2}


,accuracy,precision_macro,recall_macro,f1_macro,n_splits,mask_frac
Purpose,0.097049,0.082371,0.171000,0.102793,5.0,0.2
Employment,0.048264,0.037690,0.319562,0.065484,5.0,0.2
Job,0.155203,0.097401,0.320468,0.143638,5.0,0.2
Foreign_worker,0.180519,0.078963,0.467869,0.128446,5.0,0.2
